In [33]:
#data science - standard library
import pandas as pd
import numpy as np

# visualizations
import matplotlib.pyplot as plt
import seaborn as sns

from api_keys import g_key
import gmaps

import requests
import json

In [4]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [5]:
df = pd.read_csv("booth_cities.csv")
df.head()

,City,Latitude,Longitude,Temperature,Feels Like,Humidity,Pressure,Wind Speed,Cloudiness
0,Khani,41.96,42.96,64.40,60.37,63,1015,8.05,8.05
1,Marang,5.21,103.21,75.09,82.56,94,1010,3.36,3.36
2,Labuhan,-6.88,112.21,81.72,87.93,78,1012,6.51,6.51
3,Ushuaia,-54.80,-68.30,57.20,51.21,54,998,6.93,6.93
4,Lebu,-37.62,-73.65,50.86,46.35,78,1024,5.66,5.66


In [9]:
# Store latitude and longitude in locations
locations = df[["Latitude", "Longitude"]]

# Fill NaN values and convert to float
humids = df["Humidity"].astype(float)
locations

,Latitude,Longitude
0,41.96,42.96
1,5.21,103.21
2,-6.88,112.21
3,-54.80,-68.30
4,-37.62,-73.65
...,...,...
556,44.62,-65.77
557,11.28,-15.83
558,-0.58,117.09
559,29.03,21.55


In [17]:
coords = df[["Latitude", "Longitude"]]
humids = df.Humidity

# Customize the size of the figure
figure_layout = {
    'width': '1200px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout, map_type = "SATELLITE")

# # Assign the marker layer to a variable
# markers = gmaps.marker_layer(coords, info_box_content=states)
# # Add the layer to the map
# fig.add_layer(markers)

heat = gmaps.heatmap_layer(coords, weights = humids, dissipating=False, max_intensity=500)
# Add the layer to the map
fig.add_layer(heat)

fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

In [18]:
df.head()

,City,Latitude,Longitude,Temperature,Feels Like,Humidity,Pressure,Wind Speed,Cloudiness
0,Khani,41.96,42.96,64.40,60.37,63,1015,8.05,8.05
1,Marang,5.21,103.21,75.09,82.56,94,1010,3.36,3.36
2,Labuhan,-6.88,112.21,81.72,87.93,78,1012,6.51,6.51
3,Ushuaia,-54.80,-68.30,57.20,51.21,54,998,6.93,6.93
4,Lebu,-37.62,-73.65,50.86,46.35,78,1024,5.66,5.66


In [31]:
mask = (df.Temperature < 77) & (df.Temperature > 74) & (df["Wind Speed"] < 5)
df_sub = df.loc[mask].reset_index(drop=True)
df_sub.head()

,City,Latitude,Longitude,Temperature,Feels Like,Humidity,Pressure,Wind Speed,Cloudiness
0,Marang,5.21,103.21,75.09,82.56,94,1010,3.36,3.36
1,Wattegama,6.80,81.48,74.77,82.62,90,1010,1.12,1.12
2,Bengkulu,-3.80,102.27,76.86,82.74,82,1012,3.94,3.94
3,Itapirapuã,-15.82,-50.61,76.35,75.87,48,1010,3.74,3.74
4,Micheweni,-4.97,39.83,76.55,82.44,81,1012,3.33,3.33


In [30]:
df_sub.City.values

array(['Marang', 'Wattegama', 'Bengkulu', 'Itapirapuã', 'Micheweni',
       'Omboué', 'Airai', 'Guiratinga', 'Coroatá', 'Becerril', 'Birao',
       'Meulaboh', 'Rantauprapat', 'Abu Dhabi', 'Nioro', 'Dondo',
       'Aitape', 'Puerto Lleras', 'Santa Isabel Rio Negro'], dtype=object)

In [32]:
coords = df[["Latitude", "Longitude"]]
coords_ideal = df_sub[["Latitude", "Longitude"]]
humids = df.Humidity

# Customize the size of the figure
figure_layout = {
    'width': '1200px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout, map_type = "SATELLITE")

# Assign the marker layer to a variable
markers = gmaps.marker_layer(coords_ideal, info_box_content=df_sub.City)
# Add the layer to the map
fig.add_layer(markers)

heat = gmaps.heatmap_layer(coords, weights = humids, dissipating=False, max_intensity=500)
# Add the layer to the map
fig.add_layer(heat)

fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

In [40]:
#inits
names = []
addresses = []
# prices = []
# ratings = []

# find the closest restaurant of each type to coordinates

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "rankby": "distance",
    "type": "hotel",
    "key": g_key,
}

# use iterrows to iterate through pandas dataframe
for index, row in df_sub.iterrows():
    
    location = f"{row.Latitude}, {row.Longitude}"

    # add keyword to params dict
    params['location'] = location

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row.City}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest Hotel is {results[0]['name']}.")
        
#         types_df.loc[index, 'name'] = results[0]['name']
#         types_df.loc[index, 'address'] = results[0]['vicinity']
#         types_df.loc[index, 'price_level'] = results[0]['price_level']
#         types_df.loc[index, 'rating'] = results[0]['rating']
              
        name = results[0]['name']
        address = results[0]['vicinity']
#         price = results[0]['price_level']
#         rating = results[0]['rating']
              
        names.append(name)
        addresses.append(address)
#         prices.append(price)
#         ratings.append(rating)
              
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        names.append(np.nan)
        addresses.append(np.nan)
#         prices.append(np.nan)
#         ratings.append(np.nan)
        
    print("------------")
              
df_sub["hotel_name"] = names
df_sub["address"] = addresses
# df_sub["price_level"] = prices
# df_sub["rating"] = ratings

Retrieving Results for Index 0: Marang.
Closest Hotel is Tapak Perkhemahan Pulau Kekabu.
------------
Retrieving Results for Index 1: Wattegama.
Closest Hotel is Sugar Factory.
------------
Retrieving Results for Index 2: Bengkulu.
Closest Hotel is PT. Zikri berkah Abadi.
------------
Retrieving Results for Index 3: Itapirapuã.
Closest Hotel is Cezar consultoria agropecuaria.
------------
Retrieving Results for Index 4: Micheweni.
Closest Hotel is Duka la Dawa Shumba Mjini.
------------
Retrieving Results for Index 5: Omboué.
Closest Hotel is Medical Center Onanga Cécile.
------------
Retrieving Results for Index 6: Airai.
Closest Hotel is Raw Material Atsabe.
------------
Retrieving Results for Index 7: Guiratinga.
Closest Hotel is APAE-Associação de Pais e Amigos dos Excepcionais Gga.
------------
Retrieving Results for Index 8: Coroatá.
Closest Hotel is Comercial Santos.
------------
Retrieving Results for Index 9: Becerril.
Closest Hotel is El Corrientazo.
------------
Retrieving R

In [48]:
list_info = []

for indx, row in df_sub.iterrows():
    info = f"City: {row.City}<br>-----------<br>Hotel: {row['name']}<br>-----------<br>Address: {row.address}"
    list_info.append(info)

In [49]:
coords = df[["Latitude", "Longitude"]]
coords_ideal = df_sub[["Latitude", "Longitude"]]
humids = df.Humidity

# Customize the size of the figure
figure_layout = {
    'width': '1200px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout, map_type = "SATELLITE")

# Assign the marker layer to a variable
markers = gmaps.marker_layer(coords_ideal, info_box_content=list_info)
# Add the layer to the map
fig.add_layer(markers)

heat = gmaps.heatmap_layer(coords, weights = humids, dissipating=False, max_intensity=500)
# Add the layer to the map
fig.add_layer(heat)

fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…